# Final Project

Asaf Eliyahu & Einat Edelstien

## Creating Dataset 

We both love Elthon Joan songs, so we thought that it will be great to learn more about the lyrics of his songs.

![alt text](http://musictour.eu/data//uploads/media/albums/769/ab3e7b4194b6ec7a7dadcf43a6040eb3.jpg?raw=true)

### Collecting the Data

We search in all kinds of free lyrics web like: http://www.azlyrics.com/ and created a csv file named 'elton_john_songs_lyrics.csv' which contains all the lines of Elton john songs.

We went through each and every line of song that Elton wrote and we put each line in a different line in the csv file.

The csv file 'elton_john_songs_lyrics.csv' contained 9206 lines from Elton john song’s lyrics.

The csv file 'elton_john_songs_lyrics.csv' is our Dataset.

## Preprocessing

Before we train our model, in the preprocessing stage, we count the unique words.

We will use it later to create a mapping between words and indices, index_to_word, and word_to_index when we create the X and Y vectors of indices. These vectors are the input to the RNN. 

The following code is counting the unique words in the all songs of althon john.

In [61]:
file_to_count = open('elton_john_songs_lyrics.csv').read()
split_file = file_to_count.split()
unique_words = set(w.lower() for w in split_file)
unique_words_len = len(unique_words)
print('The number of unique words in elton john songs lyrics is ' , unique_words_len)

The number of unique words in elton john songs lyrics is  6903


In the preprocessing stage we do the following actions:

1. We first read all the sentences from the file 'elton_john.csv', and append a special SENTENCE_START token in the begining of the sentence and append a special SENTENCE_END token to each sentence. This will allow us to learn which words tend start and end a sentence.

2. We tokenize the sentences into words.

3. We count the word frequencies.

4. We get the most common words and build index_to_word and word_to_index vectors. We create a mapping between words and indices, index_to_word, and word_to_index.

5. We replace all words not in our vocabulary with the unknown token.

6. We create the training data. We create the X and Y vectors of indices. The Y vector is used to predict the next word, so the Y is just the X vector shifted by one position with the last element being the SENTENCE_END token.
These vectors are the input to the RNN.

We then use the Theano to utilize the GPU for the SGD Step.


In [67]:
import csv
import itertools
import operator
import numpy as np
import nltk
import sys
import os
import time
from datetime import datetime
from utils import *
from rnn_theano import RNNTheano

_VOCABULARY_SIZE = int(os.environ.get('VOCABULARY_SIZE', '5000'))
_HIDDEN_DIM = int(os.environ.get('HIDDEN_DIM', '80'))
_LEARNING_RATE = float(os.environ.get('LEARNING_RATE', '0.005'))
_NEPOCH = int(os.environ.get('NEPOCH', '20'))
_MODEL_FILE = os.environ.get('MODEL_FILE')

vocabulary_size = _VOCABULARY_SIZE
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

# Read the data and append SENTENCE_START and SENTENCE_END tokens
#print "Reading CSV file..."
with open('data/elton_john.csv', 'rb') as f:
    reader = csv.reader(f, skipinitialspace=True)
    reader.next()
    # Split full comments into sentences
    sentences = itertools.chain(*[nltk.sent_tokenize(x[0].decode('utf-8').lower()) for x in reader])
    # Append SENTENCE_START and SENTENCE_END
    sentences = ["%s %s %s" % (sentence_start_token, x, sentence_end_token) for x in sentences]
print ("Parsed %d sentences." % (len(sentences)))
    
# Tokenize the sentences into words
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]

# Count the word frequencies
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
#print "Found %d unique words tokens." % len(word_freq.items())

# Get the most common words and build index_to_word and word_to_index vectors
vocab = word_freq.most_common(vocabulary_size-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

#print "Using vocabulary size %d." % vocabulary_size
#print "The least frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[-1][0], vocab[-1][1])

# Replace all words not in our vocabulary with the unknown token
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]

# Create the training data
X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized_sentences])

model = RNNTheano(vocabulary_size, hidden_dim=_HIDDEN_DIM)
t1 = time.time()
model.sgd_step(X_train[10], y_train[10], _LEARNING_RATE)
t2 = time.time()
print ("SGD Step time: %f milliseconds" % ((t2 - t1) * 1000.))

if _MODEL_FILE != None:
    load_model_parameters_theano(_MODEL_FILE, model)

train_with_sgd(model, X_train, y_train, nepoch=_NEPOCH, learning_rate=_LEARNING_RATE)

num_sentences = 500
senten_min_length = 6

for i in range(num_sentences):
    sent = []
    # We want long sentences, not sentences with one or two words
    while len(sent) < senten_min_length:
        sent = generate_sentence(model)
    print (" ".join(sent))


AttributeError: '_csv.reader' object has no attribute 'next'

## Training Data

Here we start to train our model. The function 'train_with_sgd' is the function which responsible for training the model.
The inputs for the function are: 

- model: The RNN model
- X_train
- y_train
- nepoch: Number of times to iterate through the complete dataset
- learning_rate: Initial learning rate for SGD

In [68]:
def train_with_sgd(model, X_train, y_train, learning_rate=0.005, nepoch=1, evaluate_loss_after=5):
    # We keep track of the losses so we can plot them later
    losses = []
    num_examples_seen = 0
    for epoch in range(nepoch):
        # Optionally evaluate the loss
        print()
        if (epoch % evaluate_loss_after == 0):
            loss = model.calculate_loss(X_train, y_train)
            losses.append((num_examples_seen, loss))
            time = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
            print ("%s: Loss after num_examples_seen=%d epoch=%d: %f" % (time, num_examples_seen, epoch, loss))
            # Adjust the learning rate if loss increases
            if (len(losses) > 1 and losses[-1][1] > losses[-2][1]):
                learning_rate = learning_rate * 0.5  
                print ("Setting learning rate to %f" % learning_rate)
            sys.stdout.flush()
            # ADDED! Saving model oarameters
            save_model_parameters_theano("./data/rnn-theano-elton-%d-%d-%s.npz" % (model.hidden_dim, model.word_dim, time), model)
        # For each training example...
        for i in range(len(y_train)):
            # One SGD step
            model.sgd_step(X_train[i], y_train[i], learning_rate)
            num_examples_seen += 1

        print()

## Generating new stentences
We generate new 500 sentences with at least 6 word in each sentence and write the sentences to txt file named 'model_ejohn_lyrics.txt'.

In [59]:
def generate_sentence(model):
    # We start the sentence with the start token
    new_sentence = [word_to_index[sentence_start_token]]
    # Repeat until we get an end token
    while not new_sentence[-1] == word_to_index[sentence_end_token]:
        next_word_probs = model.forward_propagation(new_sentence)
        sampled_word = word_to_index[unknown_token]
        # We don't want to sample unknown words
        while sampled_word == word_to_index[unknown_token]:
            samples = np.random.multinomial(1, next_word_probs[-1])
            sampled_word = np.argmax(samples)
        new_sentence.append(sampled_word)
    sentence_str = [index_to_word[x] for x in new_sentence[1:-1]]
    return sentence_str

## The RNNTheano 

In [50]:
import numpy as np
#import theano as theano
#import theano.tensor as T
from utils import *
import operator

class RNNTheano:
    
    def __init__(self, word_dim, hidden_dim=100, bptt_truncate=4):
        # Assign instance variables
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        # Randomly initialize the network parameters
        U = np.random.uniform(-np.sqrt(1./word_dim), np.sqrt(1./word_dim), (hidden_dim, word_dim))
        V = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (word_dim, hidden_dim))
        W = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (hidden_dim, hidden_dim))
        # Theano: Created shared variables
        self.U = theano.shared(name='U', value=U.astype(theano.config.floatX))
        self.V = theano.shared(name='V', value=V.astype(theano.config.floatX))
        self.W = theano.shared(name='W', value=W.astype(theano.config.floatX))      
        # We store the Theano graph here
        self.theano = {}
        self.__theano_build__()
    
    def __theano_build__(self):
        U, V, W = self.U, self.V, self.W
        x = T.ivector('x')
        y = T.ivector('y')
        def forward_prop_step(x_t, s_t_prev, U, V, W):
            s_t = T.tanh(U[:,x_t] + W.dot(s_t_prev))
            o_t = T.nnet.softmax(V.dot(s_t))
            return [o_t[0], s_t]
        [o,s], updates = theano.scan(
            forward_prop_step,
            sequences=x,
            outputs_info=[None, dict(initial=T.zeros(self.hidden_dim))],
            non_sequences=[U, V, W],
            truncate_gradient=self.bptt_truncate,
            strict=True)
        
        prediction = T.argmax(o, axis=1)
        o_error = T.sum(T.nnet.categorical_crossentropy(o, y))
        
        # Gradients
        dU = T.grad(o_error, U)
        dV = T.grad(o_error, V)
        dW = T.grad(o_error, W)
        
        # Assign functions
        self.forward_propagation = theano.function([x], o)
        self.predict = theano.function([x], prediction)
        self.ce_error = theano.function([x, y], o_error)
        self.bptt = theano.function([x, y], [dU, dV, dW])
        
        # SGD
        learning_rate = T.scalar('learning_rate')
        self.sgd_step = theano.function([x,y,learning_rate], [], 
                      updates=[(self.U, self.U - learning_rate * dU),
                              (self.V, self.V - learning_rate * dV),
                              (self.W, self.W - learning_rate * dW)])
    
    def calculate_total_loss(self, X, Y):
        return np.sum([self.ce_error(x,y) for x,y in zip(X,Y)])
    
    def calculate_loss(self, X, Y):
        # Divide calculate_loss by the number of words
        num_words = np.sum([len(y) for y in Y])
        return self.calculate_total_loss(X,Y)/float(num_words)   


def gradient_check_theano(model, x, y, h=0.001, error_threshold=0.01):
    # Overwrite the bptt attribute. We need to backpropagate all the way to get the correct gradient
    model.bptt_truncate = 1000
    # Calculate the gradients using backprop
    bptt_gradients = model.bptt(x, y)
    # List of all parameters we want to chec.
    model_parameters = ['U', 'V', 'W']
    # Gradient check for each parameter
    for pidx, pname in enumerate(model_parameters):
        # Get the actual parameter value from the mode, e.g. model.W
        parameter_T = operator.attrgetter(pname)(model)
        parameter = parameter_T.get_value()
        print ("Performing gradient check for parameter %s with size %d." % (pname, np.prod(parameter.shape)))
        # Iterate over each element of the parameter matrix, e.g. (0,0), (0,1), ...
        it = np.nditer(parameter, flags=['multi_index'], op_flags=['readwrite'])
        while not it.finished:
            ix = it.multi_index
            # Save the original value so we can reset it later
            original_value = parameter[ix]
            # Estimate the gradient using (f(x+h) - f(x-h))/(2*h)
            parameter[ix] = original_value + h
            parameter_T.set_value(parameter)
            gradplus = model.calculate_total_loss([x],[y])
            parameter[ix] = original_value - h
            parameter_T.set_value(parameter)
            gradminus = model.calculate_total_loss([x],[y])
            estimated_gradient = (gradplus - gradminus)/(2*h)
            parameter[ix] = original_value
            parameter_T.set_value(parameter)
            # The gradient for this parameter calculated using backpropagation
            backprop_gradient = bptt_gradients[pidx][ix]
            # calculate The relative error: (|x - y|/(|x| + |y|))
            relative_error = np.abs(backprop_gradient - estimated_gradient)/(np.abs(backprop_gradient) + np.abs(estimated_gradient))
            # If the error is to large fail the gradient check
            if relative_error > error_threshold:
                print ("Gradient Check ERROR: parameter=%s ix=%s" % (pname, ix))
                print ("+h Loss: %f" % gradplus)
                print ("-h Loss: %f" % gradminus)
                print ("Estimated_gradient: %f" % estimated_gradient)
                print ("Backpropagation gradient: %f" % backprop_gradient)
                print ("Relative Error: %f" % relative_error)
                return 
            it.iternext()
        print ("Gradient check for parameter %s passed." % (pname))

In [42]:
import numpy as np

def softmax(x):
    xt = np.exp(x - np.max(x))
    return xt / np.sum(xt)

def save_model_parameters_theano(outfile, model):
    U, V, W = model.U.get_value(), model.V.get_value(), model.W.get_value()
    np.savez(outfile, U=U, V=V, W=W)
    print ("Saved model parameters to %s." % outfile)
   
def load_model_parameters_theano(path, model):
    npzfile = np.load(path)
    U, V, W = npzfile["U"], npzfile["V"], npzfile["W"]
    model.hidden_dim = U.shape[0]
    model.word_dim = U.shape[1]
    model.U.set_value(U)
    model.V.set_value(V)
    model.W.set_value(W)
    print ("Loaded model parameters from %s. hidden_dim=%d word_dim=%d" % (path, U.shape[0], U.shape[1]))

## Finding the Similarity

The following code compare between the original sentence and the generated sentences. 

We use cosine_similarity function to find the similarity between the original sentence and the generated sentences.

The txt file 'original_ejohn_lyrics.txt' contains the original sentences of elton john songs.

The txt file 'model_ejohn_lyrics.txt' contains the generated sentences created by the model. 

In [72]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# original_ejohn_lyrics

original_ejohn_lyrics_sentences = open('original_ejohn_lyrics.txt','r').read().split('\n')
#print(original_ejohn_lyrics_sentences)

# model_ejohn_lyrics

model_ejohn_lyrics_sentences = open('model_ejohn_lyrics.txt','r').read().split('\n')
#print(model_ejohn_lyrics_sentences)


v = TfidfVectorizer()
v.fit(original_ejohn_lyrics_sentences)

Y = v.transform(original_ejohn_lyrics_sentences)

sum = 0
for sentence in model_ejohn_lyrics_sentences:
    X = v.transform([sentence])
    print(sentence)
    print(original_ejohn_lyrics_sentences[cosine_similarity(X, Y).argmax()])
    print(' ')
    sum = cosine_similarity(X, Y).max()


similarity = sum/500
print('The similarity between the original and model is ' , similarity)

empty in again as the open sunlight
Gonna miss the sunlight
 
heart be make n't a n't diamante n't time
And rather all this than those diamante lovers
 
n't gone to do your little your was
I was here and I was gone
 
oh dolly n't everything of night
In and out of everything
 
vast believed , up to now
To think that I believed in you at all
 
motor in the strength of night dead suzie town
That keeps my motor running
 
in a this his your mathematics
The simple mathematics making up the map
 
blue ever to start to love
I love blue eyes
 
smile can better in i n't passing
With a smile
 
they in n't and my woods
Should I make my way out of my home in the woods"
 
reckless wire i time your moving
Left me reckless and abandoned
 
black lower in n't n't clear
You cook much better on a lower flame
 
it n't cotton 'll so n't
From the times of King Cotton
 
come all n't are of else
When are you gonna come down
 
some freefalling oh in the running
I'm freefalling into a dream
 
sing in n't done n'

Here you can see the original sentence and the most similar generated sentence.

We can learn on the similarity between the original sentences of elton john songs and the generated sentences created by the model.

![alt text](http://i.telegraph.co.uk/multimedia/archive/02660/eltonJohn_2660332b.jpg?raw=true)

## Conclusion

This was a very intensive project and we learn a lot!
We had to learn a new language (python) and combine all the tools we acquired in the class within it.
As we mentioned before, we wanted to test and see if a neuron network can learn sentence from songs specifically Elton Joan songs which we really love.

As you can see the similarity index in our result was not higher only 0.0009, but this can be explained by the following reasons.
We conclude that the reason behind it is because we didn't run the RNN with a lot of iterations. You can see in the code above that the variable "NEPOCH" equals only 20.

At the beginning we tried to run the RNN while the variable "NEPOCH" equals to 5 and to 10 until we stopped at 20. We saw that as this variable is getting bigger the similarity index is getting higher and we receive better results. 

The reason we didn’t run the RNN with a lot of iterations (higher than 20) was mainly hardware. Our computer are not strong enough and several time we had blue screens and the CPU was sky rocketing. (It was steady on 98-100% even with only 20 runs and it took a lot of time also).
The hardware in the university were even lower than our computers.

Another issue was the graphic card drivers. Using THEANO the right way meaning using GPU but we couldn't do it. we tried several drivers but nothing change still the CPU utilization was quite high.
This is a common problem with this package and after reading a lot online we figure out our graphic cards aren't suitable.

Another way we try to receive a better results was by running the code in amazon but it didn't work. The configuration was a mess and we lack the experience working with it. 

We also learned that a change in the size of the vocabulary does effect the results we get and the similarity index. Although we had poor result, the limited size of the vocabulary (about 5000 words) gives us mitigated performance. 
We had to limit the size of words in order to have a better results.

We learned that as the size of the vocabulary get bigger, the time to learn is growing and vice versa.

We think that another reason for the poor results is the data we created - we focused on lines from songs. Each line in the CSV file represent a line from a song, but this is not a homogeneous data.
Is it safe to assume that lines of a song by the same song writer have the same context?
Well after seeing the result the answer might be NO. 
But maybe a good way to test it further is to try finding similarity by those method which we thought about:
1. Test songs within the same album
2. Make each line in the data as a complete song and not only a line from a song which is lack of context.

It was interesting to look at the semantic dependencies of sentences from elton john songs. We enjoyed learning about RNN and machine learning and also creating our own dataset from scratch.   